In [ ]:
#Imports
import time
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
import os
import urllib.request
from urllib.request import urlopen
import numpy as np
import pandas as pd
from pandas import DataFrame
from requests.exceptions import ConnectionError

In [ ]:
internal_urls = set()

In [ ]:
def is_url_valid(url):
   #Checks if the url is valid or not
    parsedURL = urlparse(url)
    return bool(parsedURL.netloc) and bool(parsedURL.scheme)

In [ ]:
#The function gives all urls
count = 0
def get_all_urls(url):
    urls = set()
    # domain name of the URL without the protocol (umd.edu in this case)
    domain_name = urlparse(url).netloc
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    for a_tag in soup.findAll("a"):
        href = a_tag.attrs.get("href")
        if href == "" or href is None:
            #href is empty and we don't need that a element
            continue
        #if the link is not absolute, make it by joining relative to the base
        href = urljoin(url, href)
        parsed_href = urlparse(href)
        #constructing an absolute URL from parsed data
        href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
        if not is_url_valid(href):
            #in valid url
            continue
        if href in internal_urls:
            #it is already in the set, so we don't need to add
            continue
        if domain_name not in href:
            #it is an external link. i.e
            # Check if it is already there 
            if href not in external_urls:
#                 print(f"[EXT] External link: {href}")
                external_urls.add(href)
            continue
#         print(".",end=" ")
        global count
        count = count+1
        print(f"{count}[INT] Internal link: {href}")

        urls.add(href)
        internal_urls.add(href)
    return urls

In [ ]:
total_urls_visited = 0
def crawl(url, max_urls=30000):
    #Max URL is just to decrease the time if there are a lot of pages.
    #The following code was openly available of github and I found this
    #idea useful to inhibit crawling time
    global total_urls_visited
    total_urls_visited += 1
    try:
        links = get_all_urls(url)
        for link in links:
            if total_urls_visited > max_urls:
                break
            crawl(link, max_urls=max_urls)
    except Exception as e:
        print(e)

In [ ]:
#base_url = "https://umd.edu/virusinfo"
#Input
# base_url = input("Enter the URL : ")
base_url = "https://www.archanaskitchen.com/"
internal_urls.add(base_url)

In [ ]:
parsedurl = urlparse(base_url)
print(parsedurl)
base_url_main = parsedurl.scheme+"://"+parsedurl.netloc
print(base_url_main)
# base_url_text = base_url.split("//",1)[1]
base_url_text = parsedurl.netloc+parsedurl.path
print(base_url_text)
# base_url_text_domain = base_url_text.split("/",1)[0]
base_url_text_domain = parsedurl.netloc
print(base_url_text_domain)

In [ ]:
crawl(base_url)
print("[+] Total External links:", len(external_urls))
print("[+] Total Internal links:", len(internal_urls))
print("[+] Total:", len(external_urls) + len(internal_urls))

In [ ]:
#Converting List to Column
df = DataFrame(internal_urls,columns=['URL'])

In [ ]:
dfrecipe = df[df['URL'].str.contains('recipe')]

In [ ]:
dfrecipe.to_csv('filename.csv')